# AIML CA2

## Import General Dependencies

In [ ]:
# Data Manipulation Dependencies
import numpy as np
import pandas as pd

# Graphing Dependencies
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Miscellaneous Dependencies
from typing import Union, List, Tuple, Dict, Callable
import pickle
import warnings

In [ ]:
# Hide all warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from numpy.linalg import LinAlgError

warnings.filterwarnings(action='ignore')
warnings.simplefilter(action='ignore', category=ConvergenceWarning)

# Part A > Time Series Regression

## Import Exclusive Dependencies

In [ ]:
# Time Series Tools
from statsmodels.tsa.statespace.tools import diff
from statsmodels.tsa.seasonal import seasonal_decompose

from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Time Series Models
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX

# For data cleaning
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_squared_error

## Import Data

In [ ]:
df = pd.read_csv(filepath_or_buffer='./data/train.csv', sep=',', header=0)
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df.set_index(keys='Date', inplace=True)
df.sort_index(inplace=True)
df

## Exploratory Data Analysis

### Data Cleaning Functions

In [ ]:
def check_for_missing_values(df: pd.DataFrame):
    df_tmp = df.groupby(by=[df.index.year, df.index.quarter]).count()
    df_tmp.index.set_names(names=['Year', 'Quarter'], inplace=True)
    print(df_tmp, end='\n\n')

    complete_date_range = pd.date_range(start=df.index[0], end=df.index[-1], freq='D')
    print(f'Number of missing rows: {4 * len(complete_date_range) - df.shape[0]}')

check_for_missing_values(df=df)

In [ ]:
def groupby_date(df: pd.DataFrame, freq: str = 'D'):
    df_tmp = df.copy()
    return df_tmp.resample(rule=freq).mean()

In [ ]:
def drop_outliers(df: pd.DataFrame, gas: str, columns: List[str], band: float = 1.5):
    df_tmp = df.copy()
    outliers = {}
    for c in columns:
        vals = df[c].values
        Q1 = np.quantile(vals, q=0.25)
        Q3 = np.quantile(vals, q=0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - band * IQR
        upper_bound = Q3 + band * IQR

        df_tmp[c][(df[c] < lower_bound) | (df[c] > upper_bound)] = np.nan
        print(f'Dropped {df_tmp[c][(df[c] < lower_bound) | (df[c] > upper_bound)].shape[0]} values for {gas}: {c} (outliers)')

        outliers[c] = {
            'upper_bound': upper_bound,
            'lower_bound': lower_bound
        }
    return df_tmp, outliers

In [ ]:
class DataCleaner(BaseEstimator, TransformerMixin):
    def __init__(self, boundaries):
        self.boundaries = boundaries
    
    def fit(self, X: pd.DataFrame):
        return self

    def transform(self, X: pd.DataFrame):
        X_tmp = X.copy()
        for col in ['T', 'RH']:
            current_col = X_tmp[col]
            X_tmp[col][(current_col < self.boundaries[col]['lower_bound']) | (current_col > self.boundaries[col]['upper_bound'])] = np.nan
        return X_tmp.interpolate(method='time')

In [ ]:
def impute_missing_values(df: pd.DataFrame, gas: str, columns: List[str], method: str = 'time'):
    missing_values_count = df.isna().sum()
    for c in columns:
        print(f'Inserted {missing_values_count[c]} values for {gas}: {c} (missing)')
    return df.interpolate(method=method)

In [ ]:
def get_df_partitioned_by_gas(df: pd.DataFrame, clean_columns: List[str] = None):
    gas_dict = {}
    gas_boundaries = {}
    for gas in df['Gas'].unique():
        df_tmp = df[df['Gas'] == gas]
        if clean_columns is not None:
            df_tmp, outliers = drop_outliers(df_tmp, gas, columns=clean_columns)
            df_tmp = impute_missing_values(df_tmp, gas, columns=clean_columns)
            gas_boundaries[gas] = outliers
        df_tmp['Gas'] = [gas] * df_tmp.shape[0]
        gas_dict[gas] = df_tmp
    if clean_columns is not None:
        return gas_dict, gas_boundaries
    return gas_dict

df_partitioned = get_df_partitioned_by_gas(df)

### Before Data Cleaning

In [ ]:
def get_gas_distributions(df: pd.DataFrame):
    counts = df.groupby(by='Gas').count().min(axis=1).astype(int)
    ax = sns.barplot(x=counts.index, y=counts, palette='deep')
    ax.set_title('Counts of each gas')

get_gas_distributions(df)

In [ ]:
def get_gas_means_and_medians(df):
    df_tmp = df.groupby(by='Gas', as_index=False)
    aggs = ['Mean', 'Median']
    comb_df = pd.DataFrame()
    for i, frame in enumerate([df_tmp.mean(), df_tmp.median()]):
        tmp_df = frame
        tmp_df['Type'] = aggs[i]
        comb_df = pd.concat(objs=(comb_df, tmp_df), axis=0)
    ax = sns.barplot(data=comb_df, x='Gas', y='Value', hue='Type', palette='rainbow')
    ax.set_title('Mean and Median of each gas')

get_gas_means_and_medians(df)

# The values for the gases seem positively skewed as mean < median

In [ ]:
def get_exog_variable_distributions(df: pd.DataFrame):
    df_tmp = df.copy()
    fig, ax = plt.subplots(ncols=2, figsize=(10, 4))
    ax1 = sns.histplot(x=df_tmp['T'], kde=True, ax=ax[0])
    ax1.set_title('Temperature Distribution')
    ax2 = sns.histplot(x=df_tmp['RH'], kde=True, ax=ax[1])
    ax2.set_title('Relative Humidity Distribution')

get_exog_variable_distributions(df_partitioned['CO'])

# Temperature and Relative Humidity are positively skewed

In [ ]:
def get_gas_value_distributions(df_dict: Dict[str, pd.DataFrame]):
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 6))
    for i, gas in enumerate(df_dict.keys()):
        current_ax = ax[i // 2, i % 2]
        current_ax.set_title(gas)
        sns.histplot(data=df_dict[gas]['Value'], kde=True, ax=current_ax)
    fig.tight_layout()

get_gas_value_distributions(df_partitioned)

# The values for all the gases are positively skewed

In [ ]:
def get_general_trends(df_dict: Dict[str, pd.DataFrame], ma_windows: List[int] = [1, 7, 30], kind: str = 'group'):
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12, 7))
    for i, gas in enumerate(df_dict.keys()):
        df = df_dict[gas]
        current_ax = ax[i //2, i % 2]
        for w in ma_windows:
            df_tmp = df.rolling(window=w).mean() if kind == 'ma' else groupby_date(df, freq=f'{w}D')
            current_ax.plot(df_tmp['Value'], label=f'{w}-day {kind}')
        current_ax.set_title(gas)
        default_xticks = current_ax.get_xticks()
        current_ax.set_xticks([default_xticks[j] for j in range(len(default_xticks)) if j % 3 == 0])
    ax[0, 0].legend()

get_general_trends(df_dict=df_partitioned, ma_windows=[1, 7, 30], kind='ma')

# There are a lot of outliers

In [ ]:
def get_plots_by_gas(df_dict: pd.DataFrame):
    def get_plot(df: pd.DataFrame, ax):
        ax.plot(df['T'], label='T')
        ax.plot(df['RH'], label='RH')
        ax.plot([], [], color='g', label='Value')

        ax2 = ax.twinx()
        ax2.plot(df['Value'], color='green')
        default_xticks = ax.get_xticks()
        ax.set_xticks([default_xticks[j] for j in range(len(default_xticks)) if j % 3 == 0])

    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12, 7))
    
    for i, gas in enumerate(df_dict.keys()):
        current_ax = ax[(i // 2, i % 2)]
        df_tmp = df_dict[gas]
        get_plot(df_tmp, ax=current_ax)
        current_ax.set_title(gas)
    
    ax[0, 0].legend(loc='lower left')
    fig.tight_layout()

get_plots_by_gas(df_dict=df_partitioned)

In [ ]:
def get_corr(df_dict: Dict[str, pd.DataFrame]):
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 8))
    for i, gas in enumerate(df_dict.keys()):
        current_ax = ax[i // 2, i % 2]
        sns.heatmap(data=df_dict[gas].corr().abs(), annot=True, cmap='bone_r', vmin=0.0, vmax=1.0, ax=current_ax)
        current_ax.set_title(gas)

get_corr(df_dict=df_partitioned)

### After Data Cleaning

In [ ]:
df_partitioned, boundaries = get_df_partitioned_by_gas(df=df, clean_columns=['T', 'RH', 'Value'])

In [ ]:
get_exog_variable_distributions(df=df_partitioned['CO'])

In [ ]:
get_gas_value_distributions(df_dict=df_partitioned)

In [ ]:
get_general_trends(df_dict=df_partitioned, ma_windows=[1, 7, 30], kind='ma')

In [ ]:
get_plots_by_gas(df_dict=df_partitioned)

In [ ]:
get_corr(df_dict=df_partitioned)

## Hyper-Parameter Tuning

### Tools

In [ ]:
def cf_summary(df: pd.DataFrame, kind: Union['pacf', 'acf'], threshold: float = 0.01, max_lag: int = 20, plot: bool = False):
    corr_fn = pacf if kind == 'pacf' else acf
    plot_fn = plot_pacf if kind == 'pacf' else plot_acf

    lag_space = min(max_lag, df.count() // 2 - 1)
    corr_scores, conf_intvs = corr_fn(df, nlags=lag_space, alpha=threshold)

    lower_conf_bound = conf_intvs[:, 0] - corr_scores
    upper_conf_bound = conf_intvs[:, 1] - corr_scores

    lag_values = np.where((corr_scores < lower_conf_bound) | (corr_scores > upper_conf_bound))[0][1:]
    best_t = lag_values[np.argmax((np.abs(corr_scores[lag_values])))] if len(lag_values) > 0 else 0

    if plot:
        ax = plot_fn(df, lags=lag_space, alpha=threshold, zero=False).get_axes()[0]
        ax.set_title(f'Significant lag values: {tuple(lag_values)} Best lag: {best_t}')
        ax.set_xticks([i for i in range(1, lag_space + 1)])

    return lag_values, best_t

cf_summary(df_partitioned['CO']['Value'], kind='pacf', threshold=0.01, plot=True)

In [ ]:
def get_seasonal_periods(df_dict: Dict[str, pd.DataFrame], plot: bool = False):
    m_dict = {}
    if plot:
        fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))
    for i, gas in enumerate(df_dict.keys()):
        df = df_dict[gas]
        seasonal_component = seasonal_decompose(df['Value']).seasonal
        _, m = cf_summary(seasonal_component, kind='acf')
        m_dict[gas] = m
        if plot:
            focus = seasonal_component['2004-04-01':'2004-04-20']
            current_ax = ax[i // 2, i % 2]
            current_ax.plot(focus.index.day, focus)
            current_ax.set_xticks(focus.index.day)
            current_ax.set_title(gas)
    return m_dict

get_seasonal_periods(df_partitioned, plot=True)

In [ ]:
def get_param_summary(df_dict: Dict[str, pd.DataFrame], threshold: float = 0.01, plot_differenced_values: bool = False):
    param_summary = {}
    if plot_differenced_values:
        fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 6))
    for i, gas in enumerate(df_dict.keys()):
        df = df_dict[gas]

        is_stationary = adfuller(df['Value'])[1] < threshold

        differenced_values = diff(df['Value'], k_diff=0 if is_stationary else 1, k_seasonal_diff=1, seasonal_periods=7)

        p_values, best_p = cf_summary(differenced_values, kind='pacf', max_lag=6, threshold=threshold)
        q_values, best_q = cf_summary(differenced_values, kind='acf', max_lag=6, threshold=threshold)

        param_summary[gas] = {
            'p_values': p_values,
            'best_p': best_p,
            'd_value': 0 if is_stationary else 1,
            'q_values': q_values,
            'best_q': best_q
        }

        if plot_differenced_values:
            current_ax = ax[i // 2, i % 2]
            current_ax.plot(differenced_values)
            current_ax.set_title(df['Gas'][0])
            current_ax.set_xticks([])
            current_ax.set_xlabel(xlabel='Differenced Values')
        
    return param_summary

get_param_summary(df_partitioned, threshold=0.01, plot_differenced_values=True)

In [ ]:
def format_selective_order(l: List[int]):
    list_tmp = [0 for _ in range(max(l))]
    for i in l:
        list_tmp[i - 1] = 1
    return tuple(list_tmp)

format_selective_order([1, 6])

In [ ]:
def get_params(df_dict: Dict[str, pd.DataFrame], threshold: float = 0.01):
    param_summary = get_param_summary(df_dict, threshold=threshold)
    orders = {}
    for i, gas in enumerate(df_dict.keys()):
        p = format_selective_order(param_summary[gas]['p_values'])
        d = param_summary[gas]['d_value']
        q = format_selective_order(param_summary[gas]['q_values'])
        orders[gas] = (p, d, q)
    return orders

get_params(df_dict=df_partitioned)

### Grid Searches

In [ ]:
def evaluate_params(df: pd.DataFrame, algo: Union['sarima', 'sarimax', 'expsmth'], init_kws: Dict, fit_kws: Dict, training_size: float = 0.75):
    from sklearn.metrics import mean_squared_error
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', ConvergenceWarning)

        # Train-Test Split
        partition_index = int(df.shape[0] * training_size)

        train = df.iloc[:partition_index,:]
        test = df.iloc[partition_index:,:]

        if algo == 'sarima':
            model = SARIMAX(endog=train['Value'], **init_kws).fit(**fit_kws)
            train_pred = model.predict(start=0, end=partition_index - 1)
            test_pred = model.forecast(steps=df.shape[0] - partition_index)
        elif algo == 'sarimax':
            exog = init_kws.pop('exog')
            model = SARIMAX(endog=train['Value'], exog=train[exog], **init_kws).fit(**fit_kws)
            train_pred = model.predict(start=0, end=partition_index - 1, exog=train[exog])
            test_pred = model.forecast(steps=df.shape[0] - partition_index, exog=test[exog])
            init_kws['exog'] = exog
        elif algo == 'expsmth':
            model = ExponentialSmoothing(endog=train['Value'], **init_kws).fit(**fit_kws)
            train_pred = model.predict(start=0, end=partition_index - 1)
            test_pred = model.forecast(steps=df.shape[0] - partition_index)

        return {
            'aic': model.aic,
            'train_rmse': mean_squared_error(train['Value'], train_pred, squared=False),
            'test_rmse': mean_squared_error(test['Value'], test_pred, squared=False)
        }

In [ ]:
def evaluate_grid_search(gs_results: Dict, weights: Tuple[int, int, int] = (0.5, 0.4, 0.1)):
    gs_best_params = {}
    for gas in gs_results.keys():
        current_results = gs_results[gas]

        score_fn = lambda x: weights[0] * x['test_rmse'] + weights[1] * x['train_rmse'] + weights[2] * x['aic']

        scores = list(map(score_fn, current_results))
        index = np.argmin(scores)

        best_params = current_results[index]
        gs_best_params[gas] = {
            'params': best_params['params'],
            'score': scores[index]
        }
    return gs_best_params

#### Grid Search: SARIMA

In [ ]:
def grid_search_sarima(df_dict: Dict[str, pd.DataFrame]):
    from numpy.linalg import LinAlgError
    gs_summary = {
        'CO': [],
        'NOx': [],
        'NMHC': [],
        'O3': []
    }
    seasonal_params = {
        'P_space': [0, 1, 2],
        'D_space': [0, 1],
        'Q_space': [0, 1, 2]
    }
    seasonal_periods = get_seasonal_periods(df_dict=df_dict)
    trend_orders = get_params(df_dict=df_dict, threshold=0.01)
    for gas in df_dict.keys():
        T = trend_orders[gas]
        m = seasonal_periods[gas]
        df = df_dict[gas]
        for trend in ['n', 'c']:
            for P in seasonal_params['P_space']:
                for D in seasonal_params['D_space']:
                    for Q in seasonal_params['Q_space']:
                        try:
                            results = evaluate_params(
                                df=df,
                                algo='sarima',
                                init_kws={
                                    'order': T,
                                    'seasonal_order': (P, D, Q, m),
                                    'trend': trend
                                },
                                fit_kws={}
                            )
                            gs_summary[gas].append({
                                'params': {
                                    'init': {
                                        'order': T,
                                        'seasonal_order': (P, D, Q, m),
                                        'trend': trend
                                    },
                                    'fit': {}
                                },
                                'aic': results['aic'],
                                'train_rmse': results['train_rmse'],
                                'test_rmse': results['test_rmse']
                            })
                        except LinAlgError:
                            continue
                        except ValueError:
                            continue
    return gs_summary

In [ ]:
# Save the model
# gs_sarima = grid_search_sarima(df_dict=df_partitioned)
# pickle.dump(obj=gs_sarima, file=open('./tmp/models/gs-sarima-results.p', 'wb'))

In [ ]:
# Load the model
gs_sarima = pickle.load(file=open('./tmp/models/gs-sarima-results.p', 'rb'))

In [ ]:
# Find best parameters for SARIMA
best_sarima = evaluate_grid_search(gs_results=gs_sarima, weights=(2, 1, 0.05))
best_sarima

#### Grid Search: SARIMAX

In [ ]:
def grid_search_sarimax_1(df_dict: Dict[str, pd.DataFrame]):
    from numpy.linalg import LinAlgError
    gs_summary = {
        'CO': [],
        'NOx': [],
        'NMHC': [],
        'O3': []
    }
    seasonal_params = {
        'P_space': [0, 1, 2],
        'D_space': [0, 1],
        'Q_space': [0, 1, 2]
    }
    seasonal_periods = get_seasonal_periods(df_dict=df_dict)
    trend_orders = get_params(df_dict=df_dict, threshold=0.01)
    for gas in df_dict.keys():
        T = trend_orders[gas]
        m = seasonal_periods[gas]
        df = df_dict[gas]
        for trend in ['n', 'c']:
            for P in seasonal_params['P_space']:
                for D in seasonal_params['D_space']:
                    for Q in seasonal_params['Q_space']:
                        try:
                            results = evaluate_params(
                                df=df,
                                algo='sarimax',
                                init_kws={
                                    'order': T,
                                    'seasonal_order': (P, D, Q, m),
                                    'trend': trend,
                                    'exog': ['RH', 'T']
                                }, fit_kws={}
                            )
                            gs_summary[gas].append({
                                'params': {
                                    'init': {
                                        'order': T,
                                        'seasonal_order': (P, D, Q, m),
                                        'trend': trend,
                                        'exog': ['RH', 'T']
                                    },
                                    'fit': {}
                                },
                                'aic': results['aic'],
                                'train_rmse': results['train_rmse'],
                                'test_rmse': results['test_rmse']
                            })
                        except LinAlgError:
                            continue
                        except ValueError:
                            continue
    return gs_summary

In [ ]:
# Save the model
# gs_sarimax_1 = grid_search_sarimax_1(df_dict=df_partitioned)
# pickle.dump(obj=gs_sarimax_1, file=open('./tmp/models/gs-sarimax-1-results.p', 'wb'))

In [ ]:
# Load the model
gs_sarimax_1 = pickle.load(file=open('./tmp/models/gs-sarimax-1-results.p', 'rb'))

In [ ]:
# Find best parameters for SARIMA of SARIMAX
best_sarimax_1 = evaluate_grid_search(gs_results=gs_sarimax_1, weights=(2, 1, 0.05))
best_sarimax_1

In [ ]:
def grid_search_sarimax_2(df_dict: Dict[str, pd.DataFrame], best_sarimax_1: Dict):
    gs_summary = {
        'CO': [],
        'NOx': [],
        'NMHC': [],
        'O3': []
    }
    for gas in df_dict.keys():
        df = df_dict[gas]
        current_best_sarimax_1 = best_sarimax_1[gas]
        default_exog = current_best_sarimax_1['params']['init'].pop('exog')
        for exog in ['T', 'RH', ['T', 'RH']]:
            try:
                results = evaluate_params(
                    df=df,
                    algo='sarimax',
                    init_kws={
                        **current_best_sarimax_1['params']['init'],
                        'exog': exog
                    },
                    fit_kws={}
                )
                gs_summary[gas].append({
                    'params': {
                        'init': {
                            **current_best_sarimax_1['params']['init'],
                            'exog': exog
                        },
                        'fit': {}
                    },
                    'aic': results['aic'],
                    'train_rmse': results['train_rmse'],
                    'test_rmse': results['test_rmse']
                })
            except LinAlgError:
                continue
        current_best_sarimax_1['params']['init'] = default_exog
    return gs_summary

In [ ]:
# Save the model
# gs_sarimax_2 = grid_search_sarimax_2(df_dict=df_partitioned, best_sarimax_1=best_sarimax_1)
# pickle.dump(obj=gs_sarimax_2, file=open('./tmp/models/gs-sarimax-2-results.p', 'wb'))

In [ ]:
# Load the model
gs_sarimax_2 = pickle.load(file=open('./tmp/models/gs-sarimax-2-results.p', 'rb'))

In [ ]:
# Find the best parameters for SARIMAX
best_sarimax = evaluate_grid_search(gs_results=gs_sarimax_2, weights=(2, 1, 0.05))
best_sarimax

#### Grid Search: Exponential Smoothing

In [ ]:
def grid_search_exp_smth(df_dict: Dict[str, pd.DataFrame]):
    gs_summary = {
        'CO': [],
        'NOx': [],
        'NMHC': [],
        'O3': []
    }
    trend_types = ['add', 'mul']
    seasonal_types = ['add', 'mul']

    seasonal_periods = get_seasonal_periods(df_dict=df_dict)
    smoothing_levels = np.linspace(0.01, 0.9, 8)

    for gas in df_dict.keys():
        for smoothing_level in smoothing_levels:
            for t in trend_types:
                for s in seasonal_types:
                    df = df_dict[gas]
                    try:
                        results = evaluate_params(
                            df=df,
                            algo='expsmth',
                            init_kws={
                                'trend': t,
                                'seasonal': s,
                                'seasonal_periods': seasonal_periods[gas]
                            },
                            fit_kws={
                                'smoothing_level': smoothing_level
                            }
                        )
                        gs_summary[gas].append({
                            'params': {
                                'init': {
                                    'trend': t,
                                    'seasonal': s,
                                    'seasonal_periods': seasonal_periods[gas]
                                },
                                'fit': {
                                    'smoothing_level': smoothing_level
                                }
                            },
                            'aic': results['aic'],
                            'train_rmse': results['train_rmse'],
                            'test_rmse': results['test_rmse']
                        })
                    except LinAlgError:
                        continue
                    except ValueError:
                        continue
    return gs_summary

In [ ]:
# Save the model
# gs_expsmth = grid_search_exp_smth(df_dict=df_partitioned)
# pickle.dump(obj=gs_expsmth, file=open('./tmp/models/gs-expsmth-results.p', 'wb'))

In [ ]:
# Load the model
gs_expsmth = pickle.load(file=open('./tmp/models/gs-expsmth-results.p', 'rb'))

In [ ]:
# Find the best parameters for ExponentialSmoothing model
best_expsmth = evaluate_grid_search(gs_results=gs_expsmth, weights=(2, 1, 0.05))
best_expsmth

In [ ]:
def compare_algorithms(best_sarima: Dict[str, Dict], best_sarimax: Dict[str, Dict], best_expsmth: Dict[str, Dict]):
    best_algorithms = {}
    for gas in ['CO', 'NOx', 'NMHC', 'O3']:
        current_best_sarima = best_sarima[gas]
        current_best_sarimax = best_sarimax[gas]
        current_best_expsmth = best_expsmth[gas]

        sarima_score = current_best_sarima['score']
        sarimax_score = current_best_sarimax['score']
        expsmth_score = current_best_expsmth['score']

        best_model = ['sarima', 'sarimax', 'expsmth'][np.argmin([sarima_score, sarimax_score, expsmth_score])]

        if best_model == 'sarima':
            model = SARIMAX
            init = current_best_sarima['params']['init']
            fit = current_best_sarima['params']['fit']
        elif best_model == 'sarimax':
            model = SARIMAX
            init = current_best_sarimax['params']['init']
            fit = current_best_sarimax['params']['fit']
        elif best_model == 'expsmth':
            model = ExponentialSmoothing
            init = current_best_expsmth['params']['init']
            fit = current_best_expsmth['params']['fit']

        best_algorithms[gas] = {
            'model': model[0] if type(model) is tuple else model,
            'init': init,
            'fit': fit
        }
    return best_algorithms

In [ ]:
# Save the results
# best_models = compare_algorithms(best_sarima, best_sarimax, best_expsmth)
# pickle.dump(obj=best_models, file=open('./tmp/models/best-models.p', 'wb'))

In [ ]:
best_models = pickle.load(file=open('./tmp/models/best-models.p', 'rb'))
best_models

## Making Predictions

### Evaluating In-Sample Predictions

In [ ]:
from sklearn.metrics import mean_squared_error

def evaluate_in_sample_predictions(df_dict: Dict[str, pd.DataFrame], models: Dict[str, Dict], training_size: float = 0.75):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', ConvergenceWarning)
        results = {}
        for gas in df_dict.keys():
            current_config = models[gas]
            algo = current_config['model']

            init_params = current_config['init']
            fit_params = current_config['fit']

            df = df_dict[gas]
            partition_index = int(df.shape[0] * training_size)

            train = df[:partition_index]
            test = df[partition_index:]

            if 'exog' in init_params:
                exog = init_params.pop('exog')
                model = algo(endog=train['Value'], exog=train[exog], **init_params).fit(**fit_params)
                train_pred = model.predict(start=0, end=partition_index - 1, exog=train[exog])
                test_pred = model.forecast(steps=df.shape[0] - partition_index, exog=test[exog])
                init_params['exog'] = exog
            else:
                model = algo(endog=train['Value'], **init_params).fit(**fit_params)
                train_pred = model.predict(start=0, end=partition_index - 1)
                test_pred = model.forecast(steps=df.shape[0] - partition_index)

            train_err = mean_squared_error(train['Value'], train_pred, squared=False)
            test_err = mean_squared_error(test['Value'], test_pred, squared=False)

            results[gas] = {
                'aic': model.aic,
                'train_true': train['Value'],
                'train_pred': train_pred,
                'train_rmse': train_err,
                'test_true': test['Value'],
                'test_pred': test_pred,
                'test_rmse': test_err
            }
        return results

In [ ]:
# Save the results
# evaluation_results = evaluate_in_sample_predictions(df_dict=df_partitioned, models=best_models)
# pickle.dump(obj=evaluation_results, file=open('./tmp/models/evaluation-results.p', 'wb'))

In [ ]:
evaluation_results = pickle.load(file=open('./tmp/models/evaluation-results.p', 'rb'))

In [ ]:
def decompose(results: Dict[str, Dict]):
    aic_values = []
    train_rmse_values = []
    test_rmse_values = []

    for gas in results.keys():
        aic_values.append(results[gas]['aic'])
        train_rmse_values.append(results[gas]['train_rmse'])
        test_rmse_values.append(results[gas]['test_rmse'])
    decomposed_results = {
        'mean_aic': sum(aic_values) / len(aic_values),
        'mean_train_rmse': sum(train_rmse_values) / len(train_rmse_values),
        'mean_test_rmse': sum(test_rmse_values) / len(test_rmse_values),
    }
    return decomposed_results

decompose(results=evaluation_results)

In [ ]:
def plot_predictions(results: Dict[str, Dict]):
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12, 7))
    for i, gas in enumerate(results.keys()):
        current_results = results[gas]
        current_ax = ax[i // 2, i % 2]
        current_ax.plot(pd.concat(objs=(current_results['train_true'], current_results['test_true']), axis=0))
        current_ax.plot(current_results['train_pred'])
        current_ax.plot(current_results['test_pred'])
        current_ax.set_title(gas)

plot_predictions(results=evaluation_results)

### Evaluating Out-of-Sample Predictions (Forecasts)

In [ ]:
def get_test_df(file_path: str):
    test_df = pd.read_csv(filepath_or_buffer=file_path, sep=',', header=0)
    test_df['Date'] = pd.to_datetime(test_df['Date'], format='%d/%m/%Y')
    test_df.set_index(keys='Date', inplace=True)
    return test_df.sort_index(ascending=True)

In [ ]:
def clean_and_partition_test_data(file_path: str = './data/test.csv'):
    test_df = get_test_df(file_path=file_path)
    test_df_partitioned = {}
    for gas in test_df['Gas'].unique():
        test_df_partitioned[gas] = DataCleaner(boundaries=boundaries[gas]).fit_transform(test_df[test_df['Gas'] == gas])
    return test_df_partitioned

In [ ]:
def evaluate_out_of_sample_predictions(df_dict: Dict[str, pd.DataFrame], params: Dict[str, Dict], test_file: str, out_file: str, segment_start: float, segment_end: float):
    test_df_dict = clean_and_partition_test_data(file_path=test_file)
    results = []
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', ConvergenceWarning)
        for gas in df_dict.keys():
            df = df_dict[gas]
            target = test_df_dict[gas]
            
            partition_index_1 = int(df.shape[0] * segment_start)
            partition_index_2 = int(df.shape[0] * segment_end)

            train = df[partition_index_1:partition_index_2]
            padding = df[partition_index_2:]

            current_config = params[gas]
            algo = current_config['model']
            init_params = current_config['init']
            fit_params = current_config['fit']

            print(f'Init Parameters:\t{init_params}')
            print(f'Fit Parameters:\t{fit_params}')

            if 'exog' in init_params.keys():
                exog = init_params.pop('exog')
                model = algo(endog=train['Value'], exog=train[exog], **init_params).fit(**fit_params)
                pred = model.forecast(
                    steps=padding.shape[0] + target.shape[0],
                    exog=pd.concat(objs=(padding[exog], target[exog]), axis=0)
                ).rename('Value')
                init_params['exog'] = exog
            else:
                model = algo(endog=train['Value'], **init_params).fit(**fit_params)
                pred = model.forecast(
                    steps=padding.shape[0] + target.shape[0]
                ).rename('Value')

            result = pd.merge(left=pred, right=target, left_index=True, right_index=True, how='inner')

            print(f'{result.shape[0]} steps forecasted', end='\n\n')

            results.append(result.set_index(keys='id')['Value'])
    
    predictions = pd.concat(objs=results, axis=0)
    predictions.sort_index(inplace=True)
    predictions.to_csv(out_file, sep=',', index_label='id')

pred_df = evaluate_out_of_sample_predictions(
    df_dict=df_partitioned,
    segment_start=0.0,
    segment_end=0.75,
    params=best_models,
    test_file='./data/test.csv',
    out_file='./out/clean.csv'
)

In [ ]:
def plot_forecasts(df_dict: Dict[str, pd.DataFrame], prediction_file: str):
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 8))
    test_df_dict = clean_and_partition_test_data()
    pred_df = pd.read_csv(prediction_file, index_col=0)
    for i, gas in enumerate(df_dict.keys()):
        df = df_dict[gas]
        target = test_df_dict[gas]
        future = pd.merge(left=target, right=pred_df, left_on='id', right_index=True, how='inner')
        
        current_ax = ax[i // 2, i % 2]
        ax_twin = current_ax.twinx()

        current_ax.plot(df['Value'])
        current_ax.plot(future['Value'])
        current_ax.set_title(gas)

plot_forecasts(df_dict=df_partitioned, prediction_file='./out/clean.csv')

### Baseline Prediction (constant model)

In [ ]:
# baseline prediction, since they are all stationary
# test_set = pd.read_csv(filepath_or_buffer='./data/test.csv', sep=',')
# pd.merge(
#     left=test_set,
#     right=df.groupby(by='Gas').mean(),
#     left_on='Gas',
#     right_index=True,
#     how='inner'
# )[['id', 'Value']].set_index(keys='id').to_csv('./out/baseline.csv', sep=',', index_label='id')
plot_forecasts(df_dict=df_partitioned, prediction_file='./out/baseline.csv')

# Part B > Clustering

## Import Exclusive Dependencies

In [ ]:
# Preprocessing Dependencies
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Clustering Algorithms
from sklearn.cluster import KMeans, DBSCAN, OPTICS, AffinityPropagation, AgglomerativeClustering

# Clustering metrics
from sklearn.metrics import silhouette_score
import plotly.io
plotly.io.renderers.default='notebook'

## Import Data

In [ ]:
df2 = pd.read_csv('./data/Mall_Customers.csv', index_col=0)
df2.head()

## Exploratory Data Analysis

In [ ]:
def rename_columns(df: pd.DataFrame):
    df_tmp = df.copy()
    df_tmp.rename(mapper={
        'Genre': 'Gender',
        'Annual Income (k$)': 'Annual Income',
        'Spending Score (1-100)': 'Spending Score'
    }, axis=1, inplace=True)
    # df_tmp['Annual Income'] = df_tmp['Annual Income'] * 1000
    return df_tmp

df2 = rename_columns(df2)
df2

In [ ]:
def get_distribution_of_each_variable(df: pd.DataFrame):
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 8))
    for i, c in enumerate(df.columns):
        var = df[c]
        kde = var.dtype.kind in 'biufc'
        sns.histplot(var, kde=kde, ax=ax[i // 2, i % 2])

get_distribution_of_each_variable(df2)

In [ ]:
def get_distribution_by_gender(df: pd.DataFrame):
    fig, ax = plt.subplots(nrows=3, figsize=(6, 6))
    for i, c in enumerate(['Age', 'Annual Income', 'Spending Score']):
        var = df[c]
        sns.kdeplot(x=var, hue=df['Gender'], ax=ax[i])
    fig.tight_layout()

get_distribution_by_gender(df2)

# It seems that there are slightly more data on female customers than male ones

In [ ]:
def get_corr_heatmap(df: pd.DataFrame):
    sns.heatmap(df.corr(), annot=True, vmin=-1.0, vmax=1.0, cmap='RdBu').set_title('Correlations between Quantitative Features')

get_corr_heatmap(df2)

In [ ]:
def get_gender_importance(df: pd.DataFrame):
    fig, ax = plt.subplots(nrows=2, figsize=(5, 5))
    sns.scatterplot(data=df, x='Annual Income', y='Spending Score', hue='Gender', ax=ax[0])
    sns.scatterplot(data=df, x='Annual Income', y='Age', hue='Gender', ax=ax[1])
    fig.tight_layout()

get_gender_importance(df=df2)

### Feature Engineering

In [ ]:
# Gender column is dropped as it is not useful in solving this clustering problem
df2.drop(columns='Gender', inplace=True)

In [ ]:
def standardize_variables(df: pd.DataFrame, cols: List[str]):
    df_tmp = df.copy()
    df_tmp[cols] = StandardScaler().fit_transform(df_tmp[cols])
    return df_tmp

df2_s = standardize_variables(df2, cols=['Age', 'Annual Income', 'Spending Score'])
df2_s

In [ ]:
get_distribution_of_each_variable(df=df2_s)

## Principal Component Analysis

In [ ]:
from typing import Union, List
# plus logic
def get_pca_results(df: pd.DataFrame):
    df_scaled = StandardScaler().fit_transform(X=df)

    pca = PCA(n_components=df_scaled.shape[1]).fit(X=df_scaled)
    header = ['Eigenvalue', 'Explained Variance', 'Cumulative Explained Variance']
    header.extend(df.columns.tolist())
    eigenvalues = pca.explained_variance_
    eigenvectors = pca.components_
    expl_var = pca.explained_variance_ratio_
    cum_expl_var = pca.explained_variance_ratio_.cumsum()
    pca_results = pd.DataFrame(
        data=np.hstack((
            eigenvalues.reshape(-1, 1),
            expl_var.reshape(-1, 1),
            cum_expl_var.reshape(-1, 1),
            eigenvectors
        )),
        columns=header,
        index=[f'PC {i + 1}' for i in range(df_scaled.shape[1])]
    )

    df_transformed = pd.DataFrame(
        data=pca.transform(df_scaled),
        index=df.index,
        columns=[f'PC {i + 1}' for i in range(df_scaled.shape[1])]
    )

    return pca_results, df_transformed

pca_results, df2_transformed = get_pca_results(df=df2_s)
pca_results

In [ ]:
def scree_plot(df: pd.DataFrame, pca: pd.DataFrame):
    with sns.axes_style(style='darkgrid'):
        ax = sns.pointplot(data=pca, x=pca.index, y=pca['Eigenvalue'])
        ax.set(
            title='Scree Plot for PCA (df2)',
            ylim=(0, 1.4)
        )
        ax.annotate(text='As there is no elbow,\nno PC should be discarded', xy=(1.75, 1.2), ha='center')
        return ax

scree_plot(df2, pca_results)

## Visual Analysis - Compare Algorithms

### KMEANS

In [ ]:
def kmeans_preview(df: pd.DataFrame, clusters: List[int] = [3, 4, 5]):
    for c in clusters:
        model = KMeans(n_clusters=c)
        data = df[['Annual Income', 'Age', 'Spending Score']]
        y_hat = model.fit_predict(X=data).astype(str)
        label = f'KMeans: {c} clusters: {silhouette_score(X=data, labels=y_hat):.2f}'
        fig = px.scatter_3d(data_frame=df, x='Annual Income', y='Age', z='Spending Score', color=y_hat, title=label)
        fig.show()

kmeans_preview(df=df2)

In [ ]:
def dbscan_preview(df: pd.DataFrame, epsilon_space = np.linspace(10, 20, 3)):
    for e in epsilon_space:
        model = DBSCAN(eps=e, min_samples=10)
        data = df[['Annual Income', 'Age', 'Spending Score']]
        y_hat = model.fit_predict(X=data).astype(str)
        label = f'DBSCAN: epsilon={e}: {silhouette_score(X=data, labels=y_hat):.2f}'
        fig = px.scatter_3d(data_frame=df, x='Annual Income', y='Age', z='Spending Score', color=y_hat, title=label)
        fig.show()

dbscan_preview(df=df2)

In [ ]:
def optics_preview(df: pd.DataFrame, max_eps_space = np.linspace(10, 20, 3)):
    for e in max_eps_space:
        model = OPTICS(max_eps=e, min_samples=10)
        data = df[['Annual Income', 'Age', 'Spending Score']]
        y_hat = model.fit_predict(X=data).astype(str)
        label = f'OPTICS: max epsilon={e}: {silhouette_score(X=data, labels=y_hat):.2f}'
        fig = px.scatter_3d(data_frame=df, x='Annual Income', y='Age', z='Spending Score', color=y_hat, title=label)
        fig.show()

optics_preview(df=df2)

In [ ]:
def agglomerative_clustering_preview(df: pd.DataFrame, clusters: List[int] = [3, 4, 5]):
    for c in clusters:
        model = AgglomerativeClustering(n_clusters=c)
        data = df[['Annual Income', 'Age', 'Spending Score']]
        y_hat = model.fit_predict(X=data).astype(str)
        label = f'Agg Clustering: {c} clusters: {silhouette_score(X=data, labels=y_hat):.2f}'
        fig = px.scatter_3d(data_frame=df, x='Annual Income', y='Age', z='Spending Score', color=y_hat, title=label)
        fig.show()

agglomerative_clustering_preview(df=df2)

In [ ]:
def affinity_propagation_preview(df: pd.DataFrame, damping_factor_space = np.linspace(0.5, 0.9, 3)):
    for d in damping_factor_space:
        model = AffinityPropagation(damping=d)
        data = df[['Annual Income', 'Age', 'Spending Score']]
        y_hat = model.fit_predict(X=data).astype(str)
        label = f'Aff Propagation: damping={d}: {silhouette_score(X=data, labels=y_hat):.2f}'
        fig = px.scatter_3d(data_frame=df, x='Annual Income', y='Age', z='Spending Score', color=y_hat, title=label)
        fig.show()

affinity_propagation_preview(df=df2)

## Numerical Optimisation - Hyperparameter Selection

In [ ]:
def get_dist_score(df: pd.DataFrame, clusters: int, plot: bool = False):
    model = KMeans(n_clusters=clusters, random_state=3).fit(X=df)
    centers = model.cluster_centers_
    y_hat = model.predict(X=df)
    df_tmp = df.copy()
    df_tmp['Class'] = y_hat
    means = []
    counts = []
    for i, c in enumerate(centers):
        class_i = df_tmp[df_tmp['Class'] == i].drop(columns='Class')
        distances = np.linalg.norm(class_i - centers[i, :], axis=1)
        means.append(distances.mean())
        counts.append(len(class_i))
    
    if plot == True:
        for i, m in enumerate(means):
            plt.plot(centers[i,0], centers[i,1], 'o', mfc='none', color='r', markersize=m * 5)
        sns.scatterplot(x=df['Annual Income'], y=df['Spending Score'], hue=y_hat)

    return np.std(means) / (sum(means) / len(means)), np.std(counts)
    # return np.std(means)

get_dist_score(df2_s, 6)

In [ ]:
def get_silhouette_score(df: pd.DataFrame, clusters: int):
    from sklearn.metrics import silhouette_score
    X = df
    y_hat = KMeans(n_clusters=clusters).fit(X=X).predict(X=X)
    return silhouette_score(X=df, labels=y_hat)

get_silhouette_score(df=df2_s, clusters=6)

In [ ]:
# type: ignore
from sklearn.metrics import silhouette_score

def get_custom_score_plot(df: pd.DataFrame, clusters: List[int] = list(range(2, 11))):
    distance_scores = [get_dist_score(df, n)[0] for n in clusters]
    count_scores = [get_dist_score(df, n)[1] for n in clusters]
    silhouette_scores = [get_silhouette_score(df, n) for n in clusters]
    
    fig, ax = plt.subplots(ncols=3, figsize=(12, 4))
    with sns.axes_style(style='darkgrid'):
        ax[0].plot(clusters, silhouette_scores, color='b')
        ax[0].set_title('Silhouette Scores')
        ax[0].set_xlabel(xlabel='No. of Clusters')

        ax[1].plot(clusters, distance_scores, color='g')
        ax[1].set_title('Distance Scores')
        ax[1].set_xlabel(xlabel='No. of Clusters')

        ax[2].plot(clusters, count_scores, color='r')
        ax[2].set_title('Count Scores')
        ax[2].set_xlabel(xlabel='No. of Clusters')

get_custom_score_plot(df=df2_s)

In [ ]:
import plotly.express as px

def get_final_plot(df: pd.DataFrame, save_model: bool = False):
    df_tmp = df.copy()
    model = KMeans(n_clusters=6, random_state=7)
    model.fit(X=df_tmp)
    if save_model:
        pickle.dump(obj=model, file=open('./out/customer-clustering-model.p', 'wb'))
    y_hat = model.predict(X=df_tmp).astype(str)
    df_tmp['Class'] = y_hat
    fig = px.scatter_3d(data_frame=df_tmp, x='Age', y='Annual Income', z='Spending Score', color='Class')
    fig.update_traces(marker=dict(size=6))
    fig.show()

get_final_plot(df2_s)

In [ ]:
get_final_plot(df2)